In [1]:
# Importing Libraries

In [2]:
import pandas as pd
import numpy as np

In [3]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [4]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [5]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [6]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [7]:

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).as_matrix()

In [8]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [9]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

In [10]:
# Configuring a session
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [11]:
# Import Keras
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

Using TensorFlow backend.


In [12]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

In [13]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [14]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [15]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

C:\Users\user\Anaconda3\envs\tensorflow_gpu\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
C:\Users\user\Anaconda3\envs\tensorflow_gpu\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()


In [16]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


- Defining the Architecture of LSTM

In [17]:
# # Initiliazing the sequential model
# model = Sequential()
# # Configuring the parameters
# model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# # Adding a dropout layer
# model.add(Dropout(0.5))
# # Adding a dense output layer with sigmoid activation
# model.add(Dense(n_classes, activation='sigmoid'))
# model.summary()

In [18]:
# # Compiling the model
# model.compile(loss='categorical_crossentropy',
#               optimizer='rmsprop',
#               metrics=['accuracy'])

In [19]:
# # Training the model
# from datetime import datetime
# print(datetime.now())
# model.fit(X_train,
#           Y_train,
#           batch_size=batch_size,
#           validation_data=(X_test, Y_test),
#           epochs=epochs)
# print(datetime.now())

In [20]:
# # Confusion Matrix
# print(confusion_matrix(Y_test, model.predict(X_test)))

In [21]:
# score = model.evaluate(X_test, Y_test)

In [22]:
# score

- With a simple 2 layer architecture we got 90.09% accuracy and a loss of 0.30
- We can further imporve the performace with Hyperparameter tuning

In [23]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import Callback

epochs = 50
batch_size = 32
n_hidden = 64

# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(100, input_shape=(timesteps, input_dim),return_sequences=True))
# Adding a dropout layer
model.add(Dropout(0.7))

model.add(LSTM(100, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.7))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


filepath="weights-improvement-model1.hdf5"
checkpoint = ModelCheckpoint(filepath, verbose=1,monitor="val_acc", save_best_only=True, mode='max')
callbacks_list = [checkpoint]

# Training the model
from datetime import datetime
print(datetime.now())
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs,callbacks=callbacks_list)
print(datetime.now())
print(confusion_matrix(Y_test, model.predict(X_test)))
score = model.evaluate(X_test, Y_test)
print(score)

W1023 23:43:42.524905  1276 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\envs\tensorflow_gpu\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1023 23:43:42.526900  1276 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\envs\tensorflow_gpu\lib\site-packages\keras\backend\tensorflow_backend.py:519: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1023 23:43:42.529892  1276 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\envs\tensorflow_gpu\lib\site-packages\keras\backend\tensorflow_backend.py:4140: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1023 23:43:42.744337  1276 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\envs\tensorflow_gpu\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.place

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128, 100)          44000     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128, 100)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 606       
Total params: 125,006
Trainable params: 125,006
Non-trainable params: 0
_________________________________________________________________
2019-10-23 23:43:43.130061
Train on 7352 samples, validate on 2947 samples
Epoch 1/50
7352/7352 [==============================] - 99s 1

In [25]:
from keras.models import load_model
model1 = load_model('weights-improvement-model1.hdf5')
print(confusion_matrix(Y_test, model1.predict(X_test)))
score = model1.evaluate(X_test, Y_test)
print(score)

W1024 09:42:27.783489  2208 nn_ops.py:4224] Large dropout rate: 0.7 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W1024 09:42:27.958022  2208 nn_ops.py:4224] Large dropout rate: 0.7 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W1024 09:42:28.253257  2208 deprecation.py:323] From C:\Users\user\Anaconda3\envs\tensorflow_gpu\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 537        0         0        0                   0   
SITTING                  7      415        52        0                   0   
STANDING                 0       73       456        1                   0   
WALKING                  0        4         0      472                   5   
WALKING_DOWNSTAIRS       0        0         0        1                 418   
WALKING_UPSTAIRS         0        0         0       14                   4   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                           17  
STANDING                           2  
WALKING                           15  
WALKING_DOWNSTAIRS                 1  
WALKING_UPSTAIRS                 453  
2947/2947 [==============================] - 11s 4ms/step
[0.28

In [24]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import Callback

epochs = 300
batch_size = 32
n_hidden = 128
from keras.layers import BatchNormalization,Conv1D,Flatten,MaxPooling1D,Input,Embedding
from keras.models import Model, Sequential
from keras.optimizers import Adam, Adadelta
from keras.initializers import glorot_normal

#Configuring the parameters

inputIs = Input(shape=(timesteps, input_dim))

model = Conv1D(128, 7, activation='relu')(inputIs)
model = MaxPooling1D(3)(model)
model = Dropout(0.3)(model)

model = Conv1D(64, 7, activation='relu')(model)
model = MaxPooling1D(3)(model)
model = Dropout(0.3)(model)

model = Flatten()(model)
model = Dense(32, activation='relu')(model)
output1 = Dense(6, activation='sigmoid')(model)



model_final = Model(inputs= [inputIs], outputs=[output1])
print(model_final.summary())

# Compiling the model
model_final.compile(loss='mean_squared_error',
              optimizer=Adam(lr=0.0001),
              metrics=['accuracy'])

filepath="weights-improvement-model2.hdf5"
checkpoint = ModelCheckpoint(filepath, verbose=1,monitor="val_acc", save_best_only=True, mode='max')
callbacks_list = [checkpoint]


# Training the model
model_final.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs,callbacks=callbacks_list)
#print(confusion_matrix(Y_test, model_final.predict(X_test)))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 128, 9)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 122, 128)          8192      
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 40, 128)           0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 40, 128)           0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 34, 64)            57408     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 11, 64)            0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 11, 64)            0         
__________

W1024 09:35:53.982462  2208 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\envs\tensorflow_gpu\lib\site-packages\keras\backend\tensorflow_backend.py:988: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 7352 samples, validate on 2947 samples
Epoch 1/300
7352/7352 [==============================] - 4s 571us/step - loss: 0.1242 - acc: 0.4820 - val_loss: 0.0874 - val_acc: 0.6220

Epoch 00001: val_acc improved from -inf to 0.62199, saving model to weights-improvement-model2.hdf5
Epoch 2/300
7352/7352 [==============================] - 1s 141us/step - loss: 0.0683 - acc: 0.7146 - val_loss: 0.0665 - val_acc: 0.7492

Epoch 00002: val_acc improved from 0.62199 to 0.74924, saving model to weights-improvement-model2.hdf5
Epoch 3/300
7352/7352 [==============================] - 1s 144us/step - loss: 0.0508 - acc: 0.8240 - val_loss: 0.0560 - val_acc: 0.7981

Epoch 00003: val_acc improved from 0.74924 to 0.79810, saving model to weights-improvement-model2.hdf5
Epoch 4/300
7352/7352 [==============================] - 1s 144us/step - loss: 0.0360 - acc: 0.8889 - val_loss: 0.0443 - val_acc: 0.8476

Epoch 00004: val_acc improved from 0.79810 to 0.84764, saving model to weights-improvement-mod

7352/7352 [==============================] - 1s 139us/step - loss: 0.0089 - acc: 0.9635 - val_loss: 0.0187 - val_acc: 0.9281

Epoch 00039: val_acc did not improve from 0.93180
Epoch 40/300
7352/7352 [==============================] - 1s 138us/step - loss: 0.0089 - acc: 0.9646 - val_loss: 0.0191 - val_acc: 0.9253

Epoch 00040: val_acc did not improve from 0.93180
Epoch 41/300
7352/7352 [==============================] - 1s 138us/step - loss: 0.0086 - acc: 0.9641 - val_loss: 0.0186 - val_acc: 0.9287

Epoch 00041: val_acc did not improve from 0.93180
Epoch 42/300
7352/7352 [==============================] - 1s 142us/step - loss: 0.0082 - acc: 0.9671 - val_loss: 0.0199 - val_acc: 0.9162

Epoch 00042: val_acc did not improve from 0.93180
Epoch 43/300
7352/7352 [==============================] - 1s 141us/step - loss: 0.0083 - acc: 0.9675 - val_loss: 0.0185 - val_acc: 0.9352

Epoch 00043: val_acc improved from 0.93180 to 0.93519, saving model to weights-improvement-model2.hdf5
Epoch 44/300
73

7352/7352 [==============================] - 1s 141us/step - loss: 0.0039 - acc: 0.9848 - val_loss: 0.0160 - val_acc: 0.9406

Epoch 00080: val_acc did not improve from 0.95080
Epoch 81/300
7352/7352 [==============================] - 1s 141us/step - loss: 0.0042 - acc: 0.9833 - val_loss: 0.0162 - val_acc: 0.9362

Epoch 00081: val_acc did not improve from 0.95080
Epoch 82/300
7352/7352 [==============================] - 1s 142us/step - loss: 0.0040 - acc: 0.9850 - val_loss: 0.0160 - val_acc: 0.9393

Epoch 00082: val_acc did not improve from 0.95080
Epoch 83/300
7352/7352 [==============================] - 1s 146us/step - loss: 0.0038 - acc: 0.9848 - val_loss: 0.0153 - val_acc: 0.9433

Epoch 00083: val_acc did not improve from 0.95080
Epoch 84/300
7352/7352 [==============================] - 1s 148us/step - loss: 0.0038 - acc: 0.9860 - val_loss: 0.0153 - val_acc: 0.9457

Epoch 00084: val_acc did not improve from 0.95080
Epoch 85/300
7352/7352 [==============================] - 1s 141us/s

7352/7352 [==============================] - 1s 140us/step - loss: 0.0023 - acc: 0.9916 - val_loss: 0.0155 - val_acc: 0.9484

Epoch 00123: val_acc did not improve from 0.95182
Epoch 124/300
7352/7352 [==============================] - 1s 140us/step - loss: 0.0022 - acc: 0.9917 - val_loss: 0.0163 - val_acc: 0.9423

Epoch 00124: val_acc did not improve from 0.95182
Epoch 125/300
7352/7352 [==============================] - 1s 141us/step - loss: 0.0021 - acc: 0.9920 - val_loss: 0.0165 - val_acc: 0.9464

Epoch 00125: val_acc did not improve from 0.95182
Epoch 126/300
7352/7352 [==============================] - 1s 141us/step - loss: 0.0021 - acc: 0.9924 - val_loss: 0.0159 - val_acc: 0.9444

Epoch 00126: val_acc did not improve from 0.95182
Epoch 127/300
7352/7352 [==============================] - 1s 141us/step - loss: 0.0022 - acc: 0.9918 - val_loss: 0.0166 - val_acc: 0.9423

Epoch 00127: val_acc did not improve from 0.95182
Epoch 128/300
7352/7352 [==============================] - 1s 14

7352/7352 [==============================] - 1s 141us/step - loss: 0.0012 - acc: 0.9955 - val_loss: 0.0153 - val_acc: 0.9471

Epoch 00166: val_acc did not improve from 0.95521
Epoch 167/300
7352/7352 [==============================] - 1s 137us/step - loss: 0.0013 - acc: 0.9954 - val_loss: 0.0164 - val_acc: 0.9464

Epoch 00167: val_acc did not improve from 0.95521
Epoch 168/300
7352/7352 [==============================] - 1s 137us/step - loss: 0.0010 - acc: 0.9963 - val_loss: 0.0151 - val_acc: 0.9518

Epoch 00168: val_acc did not improve from 0.95521
Epoch 169/300
7352/7352 [==============================] - 1s 137us/step - loss: 0.0011 - acc: 0.9970 - val_loss: 0.0166 - val_acc: 0.9474

Epoch 00169: val_acc did not improve from 0.95521
Epoch 170/300
7352/7352 [==============================] - 1s 135us/step - loss: 0.0012 - acc: 0.9963 - val_loss: 0.0172 - val_acc: 0.9440

Epoch 00170: val_acc did not improve from 0.95521
Epoch 171/300
7352/7352 [==============================] - 1s 14

7352/7352 [==============================] - 1s 141us/step - loss: 7.8840e-04 - acc: 0.9971 - val_loss: 0.0157 - val_acc: 0.9488

Epoch 00208: val_acc did not improve from 0.95691
Epoch 209/300
7352/7352 [==============================] - 1s 141us/step - loss: 5.5917e-04 - acc: 0.9982 - val_loss: 0.0152 - val_acc: 0.9484

Epoch 00209: val_acc did not improve from 0.95691
Epoch 210/300
7352/7352 [==============================] - 1s 139us/step - loss: 8.0988e-04 - acc: 0.9974 - val_loss: 0.0147 - val_acc: 0.9508

Epoch 00210: val_acc did not improve from 0.95691
Epoch 211/300
7352/7352 [==============================] - 1s 142us/step - loss: 6.4212e-04 - acc: 0.9974 - val_loss: 0.0151 - val_acc: 0.9505

Epoch 00211: val_acc did not improve from 0.95691
Epoch 212/300
7352/7352 [==============================] - 1s 142us/step - loss: 5.8723e-04 - acc: 0.9978 - val_loss: 0.0150 - val_acc: 0.9484

Epoch 00212: val_acc did not improve from 0.95691
Epoch 213/300
7352/7352 [===================

7352/7352 [==============================] - 1s 144us/step - loss: 4.7730e-04 - acc: 0.9985 - val_loss: 0.0189 - val_acc: 0.9338

Epoch 00250: val_acc did not improve from 0.95691
Epoch 251/300
7352/7352 [==============================] - 1s 141us/step - loss: 4.7993e-04 - acc: 0.9982 - val_loss: 0.0164 - val_acc: 0.9457

Epoch 00251: val_acc did not improve from 0.95691
Epoch 252/300
7352/7352 [==============================] - 1s 144us/step - loss: 3.3606e-04 - acc: 0.9990 - val_loss: 0.0161 - val_acc: 0.9488

Epoch 00252: val_acc did not improve from 0.95691
Epoch 253/300
7352/7352 [==============================] - 1s 144us/step - loss: 4.4670e-04 - acc: 0.9989 - val_loss: 0.0154 - val_acc: 0.9477

Epoch 00253: val_acc did not improve from 0.95691
Epoch 254/300
7352/7352 [==============================] - 1s 142us/step - loss: 4.6522e-04 - acc: 0.9985 - val_loss: 0.0186 - val_acc: 0.9382

Epoch 00254: val_acc did not improve from 0.95691
Epoch 255/300
7352/7352 [===================

7352/7352 [==============================] - 1s 139us/step - loss: 3.2104e-04 - acc: 0.9986 - val_loss: 0.0165 - val_acc: 0.9454

Epoch 00292: val_acc did not improve from 0.95691
Epoch 293/300
7352/7352 [==============================] - 1s 146us/step - loss: 1.3612e-04 - acc: 1.0000 - val_loss: 0.0152 - val_acc: 0.9491

Epoch 00293: val_acc did not improve from 0.95691
Epoch 294/300
7352/7352 [==============================] - 1s 141us/step - loss: 2.7974e-04 - acc: 0.9992 - val_loss: 0.0169 - val_acc: 0.9433

Epoch 00294: val_acc did not improve from 0.95691
Epoch 295/300
7352/7352 [==============================] - 1s 153us/step - loss: 2.1023e-04 - acc: 0.9992 - val_loss: 0.0164 - val_acc: 0.9464

Epoch 00295: val_acc did not improve from 0.95691
Epoch 296/300
7352/7352 [==============================] - 1s 143us/step - loss: 1.9429e-04 - acc: 0.9997 - val_loss: 0.0175 - val_acc: 0.9403

Epoch 00296: val_acc did not improve from 0.95691
Epoch 297/300
7352/7352 [===================

In [26]:
from keras.models import load_model
model2 = load_model('weights-improvement-model2.hdf5')
print(confusion_matrix(Y_test, model2.predict(X_test)))
score = model2.evaluate(X_test, Y_test)
print(score)

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 537        0         0        0                   0   
SITTING                  0      409        71        0                   0   
STANDING                 0       39       492        1                   0   
WALKING                  0        0         0      494                   1   
WALKING_DOWNSTAIRS       0        0         0        0                 419   
WALKING_UPSTAIRS         0        0         0        0                   2   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                           11  
STANDING                           0  
WALKING                            1  
WALKING_DOWNSTAIRS                 1  
WALKING_UPSTAIRS                 469  
2947/2947 [==============================] - 0s 93us/step
[0.01

In [28]:
from prettytable import PrettyTable
x = PrettyTable()
x.field_names = ["Model", "Description", "dropout", "test accuracy"]
x.add_row(["Model 1", "2 LSTM layers with 100 hidden unit", "0.7", 0.9334])
x.add_row(["Model 2", "2 convolution layer with max pooling", "0.3", 0.9569])

x.border=True
print(x)

+---------+--------------------------------------+---------+---------------+
|  Model  |             Description              | dropout | test accuracy |
+---------+--------------------------------------+---------+---------------+
| Model 1 |  2 LSTM layers with 100 hidden unit  |   0.7   |     0.9334    |
| Model 2 | 2 convolution layer with max pooling |   0.3   |     0.9569    |
+---------+--------------------------------------+---------+---------------+
